In [1]:
from theano import gpuarray
%matplotlib inline
from __future__ import print_function, division
path = "data/lesson3_HW_Kaggle/"
#path = "data/lesson3_HW_Kaggle/sample/"
import utils; reload(utils)
from utils import *
from IPython.display import FileLink

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
ERROR (theano.gpuarray): Could not initialize pygpu, support disabled
Traceback (most recent call last):
  File "/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/gpuarray/__init__.py", line 227, in <module>
    use(config.device)
  File "/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/gpuarray/__init__.py", line 214, in use
    init_dev(device, preallocate=preallocate)
  File "/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/gpuarray/__init__.py", line 73, in init_dev
    pygpu_version.fullversion)
ValueError: Your installed version of pygpu(0.6.9) is too old, please upgrade to 0.7.0 or later (but below 0.8.0)
Using Theano backend.


In [4]:
import make_train_val as mtv

In [5]:
pwd

u'/home/ubuntu/courses/deeplearning1/nbs'